In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
scale = .25

mask = list()
for i, param in enumerate(net.parameters()):
    if i<4:
        m = np.ones((param.detach().numpy()).shape)
    else:
        m = np.random.binomial(1, scale, size=(param.detach().numpy()).shape)
    mask.append(torch.tensor(m))

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001*1/scale, momentum=0.9)

In [7]:
losses = list()


for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        for p, m in zip(net.parameters(), mask):
            p.grad *= m
        
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            losses.append(running_loss / 2000)
            running_loss = 0.0


            
print('Finished Training')

np.save('loss_hist/losses_'+str(int(1/scale))+'.npy', np.array(losses))

[1,  2000] loss: 2.099
[1,  4000] loss: 1.824
[1,  6000] loss: 1.668
[1,  8000] loss: 1.602
[1, 10000] loss: 1.541
[1, 12000] loss: 1.526
[2,  2000] loss: 1.466
[2,  4000] loss: 1.451
[2,  6000] loss: 1.425
[2,  8000] loss: 1.431
[2, 10000] loss: 1.393
[2, 12000] loss: 1.411
[3,  2000] loss: 1.346
[3,  4000] loss: 1.349
[3,  6000] loss: 1.368
[3,  8000] loss: 1.344
[3, 10000] loss: 1.353
[3, 12000] loss: 1.335
[4,  2000] loss: 1.297
[4,  4000] loss: 1.305
[4,  6000] loss: 1.295
[4,  8000] loss: 1.322
[4, 10000] loss: 1.309
[4, 12000] loss: 1.306
[5,  2000] loss: 1.252
[5,  4000] loss: 1.278
[5,  6000] loss: 1.302
[5,  8000] loss: 1.270
[5, 10000] loss: 1.292
[5, 12000] loss: 1.306
[6,  2000] loss: 1.247
[6,  4000] loss: 1.274
[6,  6000] loss: 1.255
[6,  8000] loss: 1.267
[6, 10000] loss: 1.284
[6, 12000] loss: 1.278
[7,  2000] loss: 1.209
[7,  4000] loss: 1.238
[7,  6000] loss: 1.244
[7,  8000] loss: 1.287
[7, 10000] loss: 1.269
[7, 12000] loss: 1.261
[8,  2000] loss: 1.226
[8,  4000] 

In [8]:
outputs = net(inputs)
loss = criterion(outputs, labels)
loss.backward()

In [9]:
print(inputs.grad)

None


In [10]:
grads = []
for param in net.parameters():
    grads.append(param.grad.view(-1))

In [11]:
paras = []
for param in net.parameters():
    paras.append(param.view(-1))

In [12]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [13]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# # print images
# imshow(torchvision.utils.make_grid(images))
# print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [14]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [15]:
outputs = net(images)

In [16]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Predicted:   ship  ship  ship plane


In [17]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 55 %


In [18]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 43 %
Accuracy of   car : 77 %
Accuracy of  bird : 43 %
Accuracy of   cat : 19 %
Accuracy of  deer : 46 %
Accuracy of   dog : 51 %
Accuracy of  frog : 67 %
Accuracy of horse : 61 %
Accuracy of  ship : 81 %
Accuracy of truck : 62 %


In [19]:
from scipy import stats
import pickle

paras = list()
dists = list()
for param in net.parameters():
    paras.append((param.view(-1)).detach().numpy())
    weights = np.squeeze(paras[-1].flatten())
    dists.append(stats.gaussian_kde(weights))
    
pickle.dump( dists, open( "dists.pkl", "wb" ) )

In [20]:
# for p, d in zip(paras, dists):
#     x_plot = np.linspace(p.min(), p.max(), 100, endpoint=True)
#     plt.figure()
#     plt.hist(p, bins=100, density=True)
#     plt.plot(x_plot, d.pdf(x_plot))
#     plt.show()